In [ ]:
!pip install pytube pydub moviepy transformers accelerate datasets[audio] torchaudio yt-dlp
!apt update && apt install -y ffmpeg

In [ ]:
import os
import torch
from pytube import YouTube
from pydub import AudioSegment
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from tqdm import tqdm
import subprocess


# Whisper model setup
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# Function to download audio from YouTube
def download_audio_from_youtube(url, output_wav="youtube_audio.wav"):
    temp_file = "temp_audio.m4a"
    # Download best audio using yt-dlp
    subprocess.run([
        "yt-dlp", "-f", "bestaudio", "-x",
        "--audio-format", "m4a", "-o", temp_file, url
    ])
    # Convert to WAV using ffmpeg
    wav_file = "converted_audio.wav"
    subprocess.run([
        "ffmpeg", "-y", "-i", temp_file, "-ar", "16000", "-ac", "1", wav_file
    ])
    os.remove(temp_file)
    return wav_file

# Function to split audio into 30-second chunks
def split_audio(input_wav, chunk_duration_ms=30000):
    audio = AudioSegment.from_wav(input_wav)
    chunk_paths = []
    for i in range(0, len(audio), chunk_duration_ms):
        chunk = audio[i:i + chunk_duration_ms]
        chunk_name = f"chunk_{i//chunk_duration_ms}.wav"
        chunk.export(chunk_name, format="wav")
        chunk_paths.append(chunk_name)
    return chunk_paths

# Transcribe and return full text
def transcribe_chunks(chunks):
    transcript = ""
    for chunk_file in tqdm(chunks, desc="Transcribing"):
        output = pipe(chunk_file)
        transcript += output["text"].strip() + "\n"
        os.remove(chunk_file)  # clean up
    return transcript

# Save transcript to file
def save_transcript(text, filename="transcription.txt"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(text)

# Main pipeline
def process_youtube_video(youtube_url):
    audio_file = download_audio_from_youtube(youtube_url)
    chunks = split_audio(audio_file)
    transcription = transcribe_chunks(chunks)
    save_transcript(transcription)
    print("✅ Transcription saved to transcription.txt")

# 👇 Replace with your YouTube URL
youtube_link = "ADD A FULL YOUTUBE LINK HERE"
process_youtube_video(youtube_link)
